In [1]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import copy
import json
import numpy as np
import matplotlib.pyplot as plt 
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pytorch_metric_learning import losses, miners, testers, samplers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from tqdm import tqdm
import random
import wandb

In [2]:
from models.efficientnet import EffNetModel
from models.swin import SwinTransformerModel
from models.efficientnet_v2 import EffNetV2Model

In [3]:
from datasets_handlers.base_dataset import BaseImageDataset, ClassLabelsMapper
from datasets_handlers.dataset_cub import cub200_dataset
from datasets_handlers.in_shop import in_shop_dataset
from datasets_handlers.stanford_online_products import stanford_products_dataset

In [4]:
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [5]:
#stanford_train = stanford_products_dataset(split='train')
#stanford_test = stanford_products_dataset(split='test')

In [6]:
#cub200_train = cub200_dataset(split='train')
#cub200_test = cub200_dataset(split='test')

In [7]:
inshop_train = ClassLabelsMapper(in_shop_dataset(split='train'))
inshop_gallery = in_shop_dataset(split='gallery')
inshop_query = in_shop_dataset(split='query')

In [8]:
image_size = 224
train_transform = A.Compose(
    [
        #A.HorizontalFlip(p=0.5),
        #A.ImageCompression(quality_lower=99, quality_upper=100),
        #A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, border_mode=0, p=0.7),
        A.Resize(image_size, image_size),
        #A.CoarseDropout(max_height=int(image_size * 0.4), max_width=int(image_size * 0.4), max_holes=1, p=0.5),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
        ToTensorV2()
    ]
)

test_transform = A.Compose([
    A.Resize(image_size, image_size),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    ToTensorV2()
])

In [9]:
dataset_train = BaseImageDataset(inshop_train, path_prefix='data/', augmentations=train_transform)
dataset_gallery = BaseImageDataset(inshop_gallery, path_prefix='data/', augmentations=test_transform)
dataset_query = BaseImageDataset(inshop_query, path_prefix='data/', augmentations=test_transform)

In [10]:
parameters = {
    'dataset:': type(dataset_train.dataset.dataset).__name__,
    'n_epochs': 100,
    'batch_size': 64,
    'lr': 1e-5
}

In [11]:
wandb.init(project="my-test-project", entity="ilya_fedorov", config=parameters)

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)


In [12]:
train_targets = []
for x, target in inshop_train:
    train_targets.append(target)

In [13]:
loss_args = {'margin': 0.5}
loss_func = losses.TripletMarginLoss(**loss_args)
#loss_func = losses.ArcFaceLoss(embedding_size=1280, num_classes=3997)

miner = miners.TripletMarginMiner(**loss_args)

sampler = samplers.MPerClassSampler(
    train_targets, m=4, length_before_new_iter=len(dataset_train)
)

In [14]:
dataloader_train = DataLoader(
    dataset_train,
    batch_size=parameters['batch_size'],
    num_workers=8,
    pin_memory=True,
    sampler=sampler
)

In [21]:
#model = EffNetModel('efficientnet-b4').cuda()
#model = SwinTransformerModel('microsoft/swin-base-patch4-window7-224')
model = EffNetV2Model('tf_efficientnetv2_m')
model.cuda()
model = model.train()

In [22]:
wandb.config.update({"loss": type(loss_func).__name__, "model": type(model).__name__, **loss_args})

In [23]:
optimizer = torch.optim.AdamW(model.parameters(), lr=parameters['lr'])

In [24]:
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, train_embeddings, test_labels, train_labels, True
    )
    return accuracies

In [25]:
accuracy_calculator = AccuracyCalculator(include=("mean_average_precision", "precision_at_1"), k=5)

In [ ]:
for epoch in range(parameters['n_epochs']):
    print(f'Starting epoch {epoch}')
    wandb.log({"epoch": epoch})
    
    for i, (data, labels) in enumerate(tqdm(dataloader_train)):
        data = data.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        embeddings = model(data)
        hard_pairs = miner(embeddings, labels)
        loss = loss_func(embeddings, labels)
        wandb.log({"train/loss": loss})
        loss.backward()
        optimizer.step()
        
        # TODO loss averaging
        
    if epoch % 10 == 0:
        # query!
        model.eval()
        test_results = test(dataset_gallery, dataset_query, model, accuracy_calculator)
        wandb.log({"test/map@1": test_results['precision_at_1']})
        wandb.log({"test/map@5": test_results['mean_average_precision']})
        model.train()

Starting epoch 0


 25%|██████████▍                               | 62/250 [00:22<01:11,  2.63it/s]

In [21]:
test_results = test(dataset_train, dataset_train, model, accuracy_calculator)

100%|███████████████████████████████████████| 1861/1861 [01:33<00:00, 19.91it/s]


In [22]:
test_results

{'mean_average_precision': 0.8537094955024545,
 'precision_at_1': 0.8352840422495005}